In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
%matplotlib inline

### GPU 설정

In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0' :
    raise SystemError('GPU device not found')
print(f'Found GPU at: {device_name}')

Found GPU at: /device:GPU:0


In [3]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18201717129971933744
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2255906407
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12844350583930554625
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


### 데이터 읽기

In [4]:
# 사용자 데이텨(u.uer) 파일을 읽기
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('./data/u.user', sep='|', names=u_cols, encoding='latin-1')
users = users.set_index('user_id')
users.head()

,age,sex,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


In [5]:
# 영화 정보 데이터(u.item) 파일을 읽기
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL',
          'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy',
          'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
          'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv('./data/u.item', sep='|', names=i_cols, encoding='latin-1')
movies = movies.set_index('movie_id')
movies.head()

,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [6]:
# 영화 평점 데이터 (u.data) 파일을 읽기
# timestamp: 평가한 연도/날짜/시간을 숫자로 변환한 값
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('./data/u.data', sep='\t', names=r_cols, encoding='latin-1')
ratings = ratings.set_index('user_id')
ratings.head()

,movie_id,rating,timestamp
user_id,,,
196,242,3,881250949
186,302,3,891717742
22,377,1,878887116
244,51,2,880606923
166,346,1,886397596


### 인기제품 방식

In [7]:
# 베스트셀러 추천
def recom_movie1(n_items) : 
    movie_sort = movie_mean.sort_values(ascending=False)[:n_items]
    recom_movies = movies.loc[movie_sort.index]
    recommendations = recom_movies['title']
    return recommendations

movie_mean = ratings.groupby(['movie_id'])['rating'].mean()
recom_movie1(5)

movie_id
814                         Great Day in Harlem, A (1994)
1599                        Someone Else's America (1995)
1201           Marlene Dietrich: Shadow and Light (1996) 
1122                       They Made Me a Criminal (1939)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object

In [8]:
# 함수 한 줄로 만들기
def recom_movie2(n_items) : 
    return movies.loc[movie_mean.sort_values(ascending=False)[:n_items].index]['title']

### 추천 시스템의 정확도 측정

In [9]:
# 정확도 계산
def RMSE(y_true, y_pred) : 
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

In [10]:
# 영화 평점에 대한 RMSE
rmse = []
for user in set(ratings.index) : 
    y_true = ratings.loc[user]['rating']
    y_pred = movie_mean[ratings.loc[user]['movie_id']]
    accuracy = RMSE(y_true, y_pred)
    rmse.append(accuracy)
print(np.mean(rmse))

0.996007224010567


- 사용자 집단별 추천

In [11]:
# 데이터 읽어오기
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('./data/u.user', sep='|', names=u_cols, encoding='latin-1')
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL',
          'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy',
          'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
          'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv('./data/u.item', sep='|', names=i_cols, encoding='latin-1')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('./data/u.data', sep='\t', names=r_cols, encoding='latin-1')

# timestamp 제거하기
ratings = ratings.drop('timestamp', axis=1)

# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]

In [12]:
# 데이터 분리하기
X = ratings.copy()
y = ratings['user_id']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, stratify=y)

In [13]:
# 모델별 RMSE 계산하는 함수
def score(model) : 
    id_pairs = zip(X_test['user_id'], X_test['movie_id'])
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
    y_true = np.array(X_test['rating'])
    return RMSE(y_true, y_pred)

# train 데이터로 Full matrix 구하기
rating_matrix = X_train.pivot(index='user_id', columns='movie_id', values='rating')

In [14]:
print(rating_matrix)

movie_id  1     2     3     4     5     6     7     8     9     10    ...  \
user_id                                                               ...   
1          NaN   3.0   4.0   3.0   NaN   5.0   4.0   1.0   5.0   3.0  ...   
2          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
3          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
4          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
5          4.0   3.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
...        ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
939        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
940        NaN   NaN   NaN   2.0   NaN   NaN   4.0   5.0   NaN   NaN  ...   
941        5.0   NaN   NaN   NaN   NaN   NaN   4.0   NaN   NaN   NaN  ...   
942        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
943        NaN   5.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   

In [15]:
# 전체 평균으로 예측치를 계산하는 기본 모델
def best_seller(user_id, movie_id) : 
    try : 
        rating = train_mean[movie_id]
    except : 
        rating = 3.0
    
    return rating

train_mean = X_train.groupby(['movie_id'])['rating'].mean()
score(best_seller)

1.0264939817929626

In [16]:
# Full matrix를 사용자 데이터와 merge
merged_ratings = pd.merge(X_train, users)
users = users.set_index('user_id')

# gender별 평점평균 계산
g_mean = merged_ratings[['movie_id', 'sex', 'rating']].groupby(['movie_id', 'sex'])['rating'].mean()

In [17]:
# Gender 기준 추천
# gender별 평균을 예측치로 돌려주는 함수
def cf_gender(user_id, movie_id) : 
    if movie_id in rating_matrix : 
        gender = users.loc[user_id]['sex']
        if gender in g_mean[movie_id] : 
            gender_rating = g_mean[movie_id][gender]
        else : 
            gender_rating = 3.0
    else : 
        gender_rating = 3.0
    return gender_rating

score(cf_gender)

1.0371354582600443

In [18]:
# occupation별 평점평균 계산
o_mean = merged_ratings[['movie_id', 'occupation', 'rating']].groupby(['movie_id', 'occupation'])['rating'].mean()

In [19]:
# occupation 기준 추천
# occupation별 평균을 예측치로 돌려주는 함수
def cf_job(user_id, movie_id) : 
    if movie_id in rating_matrix : 
        job = users.loc[user_id]['occupation']
        if job in o_mean[movie_id] : 
            job_rating = o_mean[movie_id][job]
        else : 
            job_rating = 3.0
    else : 
        job_rating = 3.0
    return job_rating

score(cf_job)

1.1208784926613637

### 내용 기반 필터링 추천

In [21]:
movies = pd.read_csv('./data/movies_metadata.csv', encoding='latin-1',
                     low_memory=False)
movies = movies[['id', 'title', 'overview']]
movies.head(10)

,id,title,overview
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,8844,Jumanji,When siblings Judy and Peter discover an encha...
2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...
5,949,Heat,"Obsessive master thief, Neil McCauley leads a ..."
6,45325,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses..."
7,9091,Sudden Death,International action superstar Jean Claude Van...
8,710,GoldenEye,James Bond must unmask the mysterious head of ...
9,9087,The American President,"Widowed U.S. president Andrew Shepherd, one of..."


In [22]:
len(movies)

45442

In [23]:
# 데이터 전처리
movies = movies.dropna()
movies['overview'] = movies['overview'].fillna(' ')
len(movies)

44330

In [25]:
# 불용어를 english로 지정하고 tf-idf 계산하기
tfidf = TfidfVectorizer(stop_words='english')
tfdif_matrix = tfidf.fit_transform(movies['overview'])

In [28]:
# cosine 유사도 계산
cosine_sim = cosine_similarity(tfdif_matrix, tfdif_matrix)
cosine_sim = pd.DataFrame(cosine_sim, index=movies.index, columns=movies.index)

In [30]:
# index-title 뒤집기
indices = pd.Series(movies.index, index=movies['title'])

# 영화제목을 받아서 추천 영화를 돌려주는 함수
def content_recommender(title, n_of_recomm) : 
    # title에서 해당 영화 index 받아오기
    idx = indices[title]

    sim_scores = cosine_sim[idx]

    # 유사도 기준으로 정렬하고, n_of_recomm만큼 가져오기(자기자신 제외: 인덱스 0)
    sim_scores = sim_scores.sort_values(ascending=False)[1:n_of_recomm+1]

    # 영화 title 반환
    return movies.loc[sim_scores.index]['title']

# 추천받기
print(content_recommender('The Lion King', 5))
print(content_recommender('The Dark Knight Rises', 10))


34664    How the Lion Cub and the Turtle Sang a Song
9339                               The Lion King 1Â½
9101                  The Lion King 2: Simba's Pride
42806                                           Prey
25637                                 Fearless Fagan
Name: title, dtype: object
12468                                      The Dark Knight
149                                         Batman Forever
1321                                        Batman Returns
15497                           Batman: Under the Red Hood
584                                                 Batman
21179    Batman Unmasked: The Psychology of the Dark Kn...
9216                    Batman Beyond: Return of the Joker
18021                                     Batman: Year One
19778              Batman: The Dark Knight Returns, Part 1
3085                          Batman: Mask of the Phantasm
Name: title, dtype: object
